In [1]:
from maclookup import ApiClient
import sqlite3
import json

PATH="data/WiGLE_Backup.db"
net = "network"
loc = "location"

API_KEY = "at_oZ9pLFT3m66EVSyi8XPYM6rMxWYnI"
MACADDR_JSON_FILE = "data/macaddr_json"


In [2]:
# Load existing data from JSON file
with open(MACADDR_JSON_FILE, 'r') as f:
    data = json.load(f)

con = sqlite3.connect(PATH)
cur=con.cursor()

new_database = False
query = "select * from {} where type != ? and type != ?".format(net)
cur.execute(query, ('W', 'B'))
if len(cur.fetchall())!=0:
    new_database = True

query = "DELETE FROM {} WHERE type != ? and type != ?".format(net)
cur.execute(query, ('W', 'B'))

query = "select count(*) from {}".format(net)
print("The number of WiFi and Bluethooth data is",cur.execute(query).fetchone()[0])

query = "select bssid from {}".format(net)
cur.execute(query)

# Extract all the new data
rts=cur.fetchall()
rt_str=[]
for rt in rts:
    if ','.join(rt) not in list(data.keys()):
    # transfer all the rt(tuple) into string
        rt_str.append(','.join(rt))
print("new data:",len(rt_str))

con.close()

The number of WiFi and Bluethooth data is 1466
new data: 1286


In [82]:
new_database

False

In [72]:
data[','.join(rts[0])]

{'vendorDetails': {'oui': '000195',
  'isPrivate': False,
  'companyName': 'Sena Tech, Inc',
  'companyAddress': '116-23 Shinlim-dong KOREA KR',
  'countryCode': 'KR'},
 'blockDetails': {'blockFound': True,
  'borderLeft': '000195000000',
  'borderRight': '000195FFFFFF',
  'blockSize': 16777216,
  'assignmentBlockSize': 'MA-L',
  'dateCreated': '2000-11-09',
  'dateUpdated': '2015-09-27'},
 'macAddressDetails': {'searchTerm': '00:01:95:4e:21:63',
  'isValid': True,
  'virtualMachine': 'Microsoft Virtual PC / Virtual Server',
  'applications': [],
  'transmissionType': 'unicast',
  'administrationType': 'UAA',
  'wiresharkNotes': 'No details',
  'comment': ''}}

In [5]:
# get mac details of new data
json_raw_data = data
times = 1
client = ApiClient(API_KEY)
new_data={}
for rt in rt_str:
    new_data[rt]=json.loads(client.get_raw_data("00:2c:c8:a2:1d:c1", 'json').decode())
    times-=1
    if times==0:
        break
print(times)
if(len(new_data)==0):
    print("This database process completely! Today still have {} times.".format(times))

0


In [6]:
#print(new_database)
print(new_data)
print(len(new_data))

{'0b:e4:6f:6d:41:e9': {'vendorDetails': {'oui': '002CC8', 'isPrivate': False, 'companyName': 'Cisco Systems, Inc', 'companyAddress': '80 West Tasman Drive San Jose CA 94568 US', 'countryCode': 'US'}, 'blockDetails': {'blockFound': True, 'borderLeft': '002CC8000000', 'borderRight': '002CC8FFFFFF', 'blockSize': 16777216, 'assignmentBlockSize': 'MA-L', 'dateCreated': '2016-11-26', 'dateUpdated': '2016-11-26'}, 'macAddressDetails': {'searchTerm': '00:2c:c8:a2:1d:c1', 'isValid': True, 'virtualMachine': 'Not detected', 'applications': [], 'transmissionType': 'unicast', 'administrationType': 'UAA', 'wiresharkNotes': 'No details', 'comment': ''}}}
1


In [87]:
# uplate json file
json_raw_data.update(new_data)

# Write updated data back to JSON file
with open(MACADDR_JSON_FILE, 'w') as f:
    json.dump(json_raw_data, f)
    

In [88]:
print(len(json_raw_data))
with open(MACADDR_JSON_FILE, 'r') as f:
    dd=json.load(f)
print(len(dd))

180
180


In [89]:
# update database
table_names=list(json_raw_data[list(json_raw_data.keys())[0]].keys())
con = sqlite3.connect(PATH)
cur=con.cursor()
for table_key in table_names:
    # Iterate through the first dimension of the JSON
    for id_key in new_data.keys():
        # Get the values for this ID and table name
        values = [new_data[id_key][table_key][col_key] for col_key in new_data[id_key][table_key].keys()]
        values=[str(item) for item in values]
        # Insert data into the table
        #print(f"INSERT INTO {table_key} (id, {', '.join(json_raw_data[id_key][table_key].keys())}) VALUES (?, {', '.join(['?' for i in range(len(values))])})", [id_key, *values])
        cur.execute(f"INSERT INTO {table_key} (id, {', '.join(new_data[id_key][table_key].keys())}) VALUES (?, {', '.join(['?' for i in range(len(values))])})", [id_key, *values])

# Commit changes and close connection
con.commit()
con.close()

# Log

In [91]:
import logging
import datetime

# Get the current date and time
now = datetime.datetime.now()

# Configure the logging module
logging.basicConfig(filename='Log_WiGLE', level=logging.DEBUG)

# Generate a log message
logging.info(now,':Inital successfully, with {} data'.format(len(new_data)))
logging.info(now,'Totally, with {} data'.format(len(dd)))


--- Logging error ---
Traceback (most recent call last):
  File "d:\python\lib\logging\__init__.py", line 1081, in emit
    msg = self.format(record)
  File "d:\python\lib\logging\__init__.py", line 925, in format
    return fmt.format(record)
  File "d:\python\lib\logging\__init__.py", line 664, in format
    record.message = record.getMessage()
  File "d:\python\lib\logging\__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "d:\python\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "d:\python\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "d:\python\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "d:\python\lib\site-packages\traitlets\config\application.py", line 837, in launch_instance
    app.start()
  File "d:\python\lib\site-packages\ipykernel\kernelapp.py"